In [ ]:
# Extract annotations from pdf

In [ ]:
# import pprint
# from pdfminer.pdfparser import PDFParser
# from pdfminer.pdfdocument import PDFDocument
# from pdfminer.pdfpage import PDFPage
# from pdfminer.pdfpage import PDFTextExtractionNotAllowed
# from pdfminer.pdfinterp import PDFResourceManager
# from pdfminer.pdfinterp import PDFPageInterpreter
# from pdfminer.pdfdevice import PDFDevice

## Reading and parsing pdf with pdfminer.
Most of this is per pdfminer doc.

Note pdfminer documentation is fairly sparse and some big changes between versions.

examples:

https://dadruid5.wordpress.com/2014/08/14/getting-started-extracting-tables-with-pdfminer/
http://slott-softwarearchitect.blogspot.com/2012/02/pdf-reading.html

http://stackoverflow.com/questions/25248140/how-does-one-obtain-the-location-of-text-in-a-pdf-with-pdfminer

This example especially describes the functions better and in context:

http://denis.papathanasiou.org/posts/2010.08.04.post.html

For information on pdf format and keys used in pdf:

http://www.adobe.com/content/dam/Adobe/en/devnet/acrobat/pdfs/pdf_reference_1-7.pdf


In [ ]:
# fname = '../me.pdf' # has several highlights in green.
# fp = open(fname, 'rb')

Boiler plate setting up parser etc.
https://euske.github.io/pdfminer/programming.html

In [ ]:
# # Create a PDF parser object associated with the file object.
# parser = PDFParser(fp)
# # Create a PDF document object that stores the document structure.
# # Supply the password for initialization.
# document = PDFDocument(parser) #, password) if password protected.
# # Check if the document allows text extraction. If not, abort.
# if not document.is_extractable:
#     raise PDFTextExtractionNotAllowed
# # Create a PDF resource manager object that stores shared resources.
# rsrcmgr = PDFResourceManager()
# # Create a PDF device object.
# device = PDFDevice(rsrcmgr)
# # Create a PDF interpreter object.
# interpreter = PDFPageInterpreter(rsrcmgr, device)
# # Process each page contained in the document.
# for page in PDFPage.create_pages(document):
#     interpreter.process_page(page)


# my_pages = PDFPage.get_pages(fname, set([1])) # should be create_pages? get_pages deprecated?

# p1 = list(my_pages)[0:1]
# p1

In [ ]:
# parser = PDFParser(fp)
# document = PDFDocument(parser)
# if not document.is_extractable:
#     raise PDFTextExtractionNotAllowed
# rsrcmgr = PDFResourceManager()

# from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTFigure, LTChar, LTAnno
# from pdfminer.converter import PDFPageAggregator

# laparams = LAParams() # setting layout analysis params
# # Create a PDF page aggregator object.
# device = PDFPageAggregator(rsrcmgr, laparams=laparams)
# interpreter = PDFPageInterpreter(rsrcmgr, device)

### every page must be processed before we can work with it.

In [ ]:
# layouts = []
# for page in PDFPage.create_pages(document):
#     interpreter.process_page(page)
#     layouts.append(device.get_result())

In [ ]:
# pages = list(PDFPage.get_pages(fp, set()))
# print(type(pages[0]))
# #print(layouts) # not sure why we'd need this?

In [ ]:
# LTPages with rectangles and rotations
#print(layouts)

In [ ]:
# annots = pages[0].annots.resolve() # list of pdfminer.pdftypes.PDFObjRef

In [ ]:
# print(len(annots))
# print(type(annots[0]))

Note how number of annots roughly corresponds to number of blocks of highlights visible in pdf.

<img src='me_pdf.png'>

### refer to adobe documentation for annotation object keys, but here is a brief list

A highlighted annotation has several key, value items.

The most important to us are:

- T: by convention, the annotator's name.
- C: RGB color if 3 elements
- NM: annotation name
- Rect: location on page in user space units, 
- Contents: not avail here, but if avail, the text of
- Popup: if the highlight was related to a popup annotation, the popup is here.

The details on what the keys mean are spread across multiple tables in Adobe PDF Doc. This is because: 
- **highlight** annotation is a subtype of a **text markup** annotation (adds 2 fields).
- **text markup** annotations are types of **markup annotations** (adding many fields including annotator name via T.)
- **markup annotations** are types of **annotations** (several fields)

Adobe documentation Table 8.3 **highlight** is a Text Markup Annotations which adds : Subtype /Highlight, QuadPoints 

- Adobe doc Section 8.4 annotations.
- Adobe doc Table 8.15 keys definitions for all annotations.
- Adobe doc Table 8.30 keys added for text markup annotations. (highlight is a subtype of text markup annotation)
- Adobe doc Table 8.21 Additional entries to markup annotations.

text markup (table 8.30) adds only:

- subtype (we care about highlight subtype)
- quad points
- But note that AP, if present, has precedence over these quadpoints.

AP (dictionary for how annotation appears):
(This seems relevant to mouse scrollover effects and fine tuning of the bounding box of the annotations.)
- table 8.15
- section 8.4.4


quad points  : (8 x n) where each is a word (X1,Y1,X2,Y2...XX1,YY1,...) so n points / 8 = number of words

note AP takes precedence over quadpoints.

### Examining a couple annotations
Note that annot 1 seems to not be of the 'highlight' subtype. Maybe it was a stray and invisible mark?

Note that annot 0 is a 'highlight' subtype, we can see the name of the annotator with the 'T' flag.

The Quadpoints should allow us to extract the text under the annotation.

In [ ]:
#pprint.pprint(annots[0].resolve().items())
# print('-----')
# pprint.pprint(annots[1].resolve().items())

In [ ]:
#if we had pdfquery working...
# pdf.extract([
#    ('last_name', ':in_bbox("315,680,395,700")'),
#    ('year', ':contains("Form 1040A (")', lambda match: int(match.text()[-5:-1]))
#    ])
# text = pdf.pq('LTPage[page_index="0"] :in_bbox("10,10,300,300")').text()

# Begin functions for final solution

### extract text using quadpoints.

In [1]:
import pprint
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTFigure, LTChar, LTAnno
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdftypes import PDFObjRef

In [2]:
# parser = PDFParser(fp)
# document = PDFDocument(parser)
# if not document.is_extractable:
#     raise PDFTextExtractionNotAllowed
# rsrcmgr = PDFResourceManager()


# laparams = LAParams() # setting layout analysis params
# # Create a PDF page aggregator object.
# device = PDFPageAggregator(rsrcmgr, laparams=laparams)
# interpreter = PDFPageInterpreter(rsrcmgr, device)

Note, the coordinate system is from lower left (0, 0) to upper right.

And the locations are as points.

We need a little slop to capture some of the Wider and taller characters.

In [112]:
def in_bbox(bbox, bbox_target):
    """Returns True if bbox is within bbox_target
    
    x, y = 0, 0 is lower left of page
    Note, we end up adding a bit of buffer space to increase tolerance 
    for the highlighting thickness. In some cases this is still not enough.
    
    TODO: rewrite other code, if no highlight found, increase the tolerances here.
    """
    y_buff = 1.0
    bx0, by0, bx1, by1 = bbox
    x0, y0, x1, y1 = bbox_target
    retval = False
    #if bx0 >= x0 - 1 and by0 >= y0 - 1 and bx1 <= x1 + 1 and by1 <= y1 + 1:
    if bx0 >= x0 - 1 and by0 >= y0 - 1 and bx1 <= x1 + 1 and by1 <= y1 + 3: # y1 + 2 sufficient for most.
        retval = True
    return(retval)

in_bbox([98.1348, 498.413, 289.085, 507.747], [95, 400, 300, 550])

True

In [6]:
def parse_layout(layout, bbox_target):
    """Extract text within bbox.
    
    modified from:
    
    http://stackoverflow.com/questions/25248140/how-does-one-obtain-the-location-of-text-in-a-pdf-with-pdfminer
    """
    chars = []
    for lt_obj in layout:
        # print(lt_obj.__class__.__name__)
        # print(lt_obj.bbox)
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            for lt_line_obj in lt_obj:
                # print(lt_line_obj.get_text())
                # print(lt_line_obj.bbox)
                capture = False
                prev = None
                for lt_subline_obj in lt_line_obj:
                    if isinstance(lt_subline_obj, LTChar):
                        # LTAnno (spaces...) have no position
                        # if position is in quad points, print
                        capture = in_bbox(lt_subline_obj.bbox, bbox_target)
                        if capture:
                            # print('previous {}'.format(prev)) # as check if we miss a leading character to check how much slop is needed.
                            # print(lt_subline_obj.bbox)
                            chars.append(lt_subline_obj.get_text())
                        prev = lt_subline_obj.bbox
                    if isinstance(lt_subline_obj, LTAnno):
                        if capture:
                            chars.append(lt_subline_obj.get_text())
    return(chars)

In [7]:
# #parse_layout(layouts[0])
# test_quad = [98.1348, 507.747, 289.085, 507.747,
#              98.1348, 498.413, 289.085, 498.413]
# #[90, 500, 300, 510])
# corners = [4, 5, 2, 3]
# test_box = [test_quad[i] for i in corners]
# test_box

Assumption the quadrilateral is not rotated.

In [8]:
# #test_box = [51.0236, 485.997, 289.042, 495.332]
# #parse_layout(layouts[0], [105.28789118, 498.3230095, 109.74117338, 507.6579657])
# #parse_layout(layouts[0], test_box)
# testA = [98.1348, 507.747, 289.085, 507.747, 98.1348, 498.413, 289.085, 498.413]
# #testA = [98.1347, 507.747, 289.085, 507.747, 98.1347, 498.413, 289.085, 498.413]
# testA = [testA[i] for i in corners]
# parse_layout(layouts[0],testA)

In [9]:
### loop through all annotations
#for each annotation
#  parse quad points into bbox
#  for each bbox
#      append bbox characters
#  concat bbox characters.

In [10]:
from itertools import islice
def quads_to_boxes(quads):
    '''Returns bounding box from quad points.
    
    A single quadpoint is 8 positions long to specify x, y locations.
    We assume all quadpoints are defining non-rotated text and therefore 
    can be specified more simply as 4 coordinate boxes.
    
    The box returned is [x0, y0, x1, y1] where x0 and y0 are closest
    to bottom left corner of page.
    
    example quad : 
    
        [98.1348, 507.747, 289.085, 507.747,
         98.1348, 498.413, 289.085, 498.413]

    example box from this quad :
       
       (we get this from elements [4, 5, 2, 3] )
       
       [98.1348, 498.413, 289.085, 507.747]
    '''
    boxes = []
    xyxy = [4, 5, 2, 3] # corners of box from quads
    for i in range(0, len(quads), 8):
        box_idx = [j+i for j in xyxy]
        box = [quads[k] for k in box_idx]
        boxes.append(box)
    return(boxes)
    

In [11]:
def get_anno_boxes(annots):
    #test quads_to_boxes
    boxes = []
    for anno in annots:
        #TODO: make sure we have annotation type is highlight.
        anno_d = dict(anno.resolve().items())
        if 'QuadPoints' not in anno_d:
            continue
        annotator = anno_d['T']
        color = anno_d['C']
        quads = anno_d['QuadPoints']
        these_boxes = quads_to_boxes(quads)
        boxes.extend(these_boxes)
    return(boxes)

#boxes = get_anno_boxes(annots)

### Sorting by boxes is complex with 2 columns of pdf text...
Sorting by boxes is complex with 2 columns of pdf text and with annotations starting or ending anywhere in a line. In addition, lines may be slightly offset from each other in the different columns.

Together this suggests we should instead pull out our annotations by going straight through the lines and checking for annotations.

In [12]:
# boxes = sorted(boxes, key = lambda x : (-x[3], x[0]))
# highlights = []
# for box in boxes:
#     print(box)
#     print(''.join(parse_layout(layouts[0], box)))
#     highlight = ''.join(parse_layout(layouts[0], box))
#     highlight = highlight.replace('-\n', '')
#     highlight = highlight.replace('\n', ' ')
#     highlights.append(highlight)
#     # replace newline splits
# print(' '.join(highlights))

### Print regions within highlights
We have 52 bboxes that correspond to highlights.

We loop through all the text boxes in layout cleaning them up.

Then we loop through a second time extracting all our hightlighted text or sentences containing that text.

In [13]:
#bboxes = [[98.1348, 498.413, 289.085, 507.747], [51.0236, 485.997, 289.042, 495.332]]
#bbox = [51.0236, 485.997, 289.042, 495.332] # true
#bbox = [51.0236, 4.997, 19.042, 41.332] # false
def in_bboxes(bbox_target, bboxes):
    '''Returns True if the target is within any of the bboxes'''
    in_target = map(lambda b: in_bbox(bbox_target, b), bboxes)
    return(any(in_target))

In [15]:
def get_bbox_text(LTText):
    '''Gets text from LTTextBox or LTTextLine if it is within bbox.
    
    Note: these are generally LTTextBoxHorizontals'''
    bbox_text = [LTText.bbox, []]
    for line in LTText: # LTTextLineHorizontal
        for char_or_anno in line:
            if isinstance(char_or_anno, LTChar):
                char_bbox = char_or_anno.bbox
                this_bbox_char = (char_bbox, char_or_anno.get_text())
                bbox_text[1].append(this_bbox_char)
            if isinstance(char_or_anno, LTAnno): # LTAnno is spaces etc (not annotation, and no bbox)
                this_bbox_char = (char_bbox, char_or_anno.get_text())
                bbox_text[1].append(this_bbox_char)
    return(bbox_text)

In [16]:
def join_sort_bboxes(bbox_1, bbox_2):
    '''joins 2 bboxes together returning one bbox.
    
    Assumes 1 is within 2
    Takes the overall position from first bbox, 
    joins and sorts all the characters and their positoins.'''
    bbox = bbox_2[0]
    chars = bbox_1[1] + bbox_2[1]
    chars = sorted(chars, key=lambda c: (-c[0][3], c[0][0]))
    new_bbox = [bbox, chars]
    return(new_bbox)

In [17]:
def sort_all_chars_in_all_bboxes(layout):
    '''Combines chars from textboxes or textlines if one is contained 
    within other's region.
    
    This is most helpful where pdf lines are made up of multiple 
    LTTextLines such as sometimes happens at end of columns.
    
    '''
    def loop_replace(restart, all_bbox_texts):
        drop = []
        for i in range(0, len(all_bbox_texts)):
            for j in range(i + 1, len(all_bbox_texts)):
                i_in_j = in_bbox(all_bbox_texts[i][0], all_bbox_texts[j][0])
                j_in_i = in_bbox(all_bbox_texts[j][0], all_bbox_texts[i][0])
                if i_in_j:
                    new_bbox_text = join_sort_bboxes(all_bbox_texts[i], all_bbox_texts[j])
                elif j_in_i:
                    new_bbox_text = join_sort_bboxes(all_bbox_texts[j], all_bbox_texts[i])
                if i_in_j or j_in_i: # shouldn't matter if we are dropping j or i
                    all_bbox_texts[i] = new_bbox_text
                    drop.append(j)
                    restart = True
        all_bbox_texts = [all_bbox_texts[z] for z in range(0, len(all_bbox_texts)) if z not in drop]
#         print(drop)
#         print(len(all_bbox_texts))
        return(restart, all_bbox_texts)
    
    #TODO: test completeness on other orderings of contained text.
    def join_if_within(all_bbox_texts):
        '''Returns list of bbox_texts, joining together a region 
        with its encompassing region.'''
        restart = True
        n = 0
        while restart:
            n = n+1
            restart = False
            restart, all_bbox_texts = loop_replace(restart, all_bbox_texts)
        return(all_bbox_texts)
 
    i_boxes = [] #[None] * len(layout) # may not use all...
    bboxes = []
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            bboxes = get_bbox_text(lt_obj)#, bbox)
            i_boxes.append(bboxes)
    i_boxes = join_if_within(i_boxes)
    return(i_boxes)

# for b in layouts[0]:
#     print(b)

# c = sort_all_chars_in_all_bboxes(layouts[0])

In [18]:
import re

def extract_only_boxes(boxes, bbox_targets):
    """Extract text within bboxes. For example, 
    our target may be highlighted regions.
    
    Proceeds through all text, capturing if in target regions.
    
    modified from:
    
    http://stackoverflow.com/questions/25248140/how-does-one-obtain-the-location-of-text-in-a-pdf-with-pdfminer
    
    params:
        boxes [[bbox, [(bbox, char), ...]]] : a cleaned up, ordered list of textregions.
        bbox_targets [x0, y0, x1, y1] : rectangle corresponding to region we want to extract.
    """
    result = []
    # run through all characters, grabbing them if they are in bbox_targets
    for b_region in boxes:
        for ch in b_region[1]:
            if in_bboxes(ch[0], bbox_targets):
                result.append(ch[1])
            elif ch[1] == '.': # tag ends of regions, may end up with many of these
                result.append(ch[1])
            else:
                result.append(' ') # in case multiple annotations in same region make sure there is a space between them.
                                   # but side effect is introducing many space characters.
    result = ''.join(result)
    # extensive cleanup of our parsing! what a mess! could be simpler?
    result = result.strip()
    result = re.sub('[ ][ ]+', ' ', result) # introduced while we tried to keep a few necessary spaces.
    result = re.sub('( \\.){1,}', '.', result) # introduced from keeping all sentence terminators.
    result = re.sub('^. ', '', result) # clean up sentence beginnings.
    result = re.sub('([a-z])[.]+([A-Za-z])', '\\1. \\2', result) # clean up sentence boundaries
    result = re.sub(' [.]([A-Z])', '. \\1', result) # clean up sentence boundaries.
    result = re.sub('[.][.]+', '', result) # trim extra 'sentence or phrase ending' tags.
    result = result.replace('-\n', '') # not entirely safe! e.g. force-\nmatching ideally is force-matching.
    result = result.replace('\n', ' ')
    result = re.sub('[?,!:;.]+[.]', '.', result) # 'highlight ends on punct' common corner cases.
    if len(result) > 0:
        result = result + '.' # make sure there is an ending.               
        result = re.sub('[ .]?[ .]?[ .].$', '.', result) # clean sentence endings.
    return(result)

In [19]:
# test = ' . . .atest'
# test = ' . . .atest. a test. . . a test'

# test = re.sub('( \\.)+', '.', test)
# test = ' . . .atest. a test,. . . a test. .'
# #test = re.sub('( \\.)+', '.', test)
# result = re.sub('[ .]?[ .]?[ .]?$', '.', test)
# print(result)

# test =  'Thus, these alleles may affect performance particularly when sustained dopamine release is necessary..'
# result = re.sub('[ .][.]$', '.', test)
# result

In [20]:
#extract_only_boxes(c, boxes)

In [53]:
def parse_pages(pages, layouts, first_n_pages):
    results = []
    for i, p in enumerate(pages[0:first_n_pages]):
        logging.debug('Processing page : {}'.format(i))
        annots = []
        if isinstance(p.annots, PDFObjRef):
            annots = p.annots.resolve()
        elif p.annots is not None: # annotations are in a list
            logging.debug('Annotations in list...')
            for a in p.annots:
                annots.append(a)
        else:
            logging.debug('No annotations found for page : {}'.format(i))
        annot_boxes = get_anno_boxes(annots)
        # gat all the text regions from this page,
        # reordering some elements if necessary.
        c = sort_all_chars_in_all_bboxes(layouts[i])
    
        # capture all the text under the annotations.
        page_result = extract_only_boxes(c, annot_boxes)
        logging.debug(page_result)
        results.append(page_result)
    results = ' '.join(results).strip()
    return(results)

In [45]:
def parse_pdf(fname):
    '''Parse highlighted annotations out of pdf.'''
    logging.info('processing file : {}'.format(fname))

    fp = open(fname, 'rb')
    parser = PDFParser(fp)
    document = PDFDocument(parser)
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed
    rsrcmgr = PDFResourceManager()

    laparams = LAParams() # setting layout analysis params
    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # work with individual pages
    layouts = []
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        layouts.append(device.get_result())

#TODO: italics are getting sorted higher than non-italics... fix by giving extra range to sort on y?
#TODO: italics (cont.) sometimes the italicized text is moved within the line, sometimes dropped entirely.

    # get the annotation boxes for the page
    pages = list(PDFPage.get_pages(fp, set()))

    results = parse_pages(pages, layouts, first_n_pages=2)
    fp.close()
    return(results)

In [46]:
def spacy_split_sentences(text):
    sentences = []
    #doc = nlp(text.decode('utf8')) #"This is a sentence. Here's another...".decode('utf8'))
    doc = nlp(text) #"This is a sentence. Here's another...".decode('utf8'))
    for span in doc.sents:
        #sentences.append(u''.join(doc[i].string for i in range(span.start, span.end)).encode('utf-8').strip())
        sentences.append(''.join(doc[i].string for i in range(span.start, span.end)))#.strip())
    sentences = '\n'.join(sentences)
    sentences = sentences + '\n'
    return(sentences)

In [55]:
import logging
import glob
import os
import codecs
from spacy.en import English

logger = logging.getLogger() # use logger set by jupyter.
#logger.setLevel(logging.INFO)
logger.setLevel(logging.INFO)

nlp = English(parser=True)

### Tests to be sure annotations are split into maximal sentences.
Not necessarily full sentences. Just want to make sure if annotations don't include sentence terminals, we have introduced a sentence terminal as needed. 

Some of these tests are especially for result endings.

In [110]:
#%cd /Users/ccarey/Documents/Projects/NAMI/rdoc

# bold text
# print('---------------')
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/02_AR03_04_25136085_mk_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# problem document, highlights are shifted.
# print('---------------')
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/02_AR03_03_25160677_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# problem document, highlights are shifted.
# print('---------------')
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/02_AR03_01_25261920_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# #problem document, highlights are shifted, even though the preview looks ok.
# print('---------------')
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/02_AR00_03_25788679_cc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# even though my highlights look 'narrower' than others, they seem ok
#  
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR01_20695690_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)
# print('---------------')

# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR01_21699821_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# print('---------------')
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR01_21849230_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# problem with list, maybe as dict instead of list?
# ERRORS as no .resolve for dict method... anno_d = dict(anno.resolve().items())
# print('---------------')
# logger.setLevel(logging.DEBUG)
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/_02_LS00_03_25913552_jl_IRRELEVANT.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# not catching annotation. Needed increased height tolerance (+3)
# logger.setLevel(logging.DEBUG)
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/02_AP00_01_25774613_cc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

# Required much larger height tolerance (+/- 7?)
# logger.setLevel(logging.DEBUG)
test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/02_AP01_05_24333745_cc_ANNOTATED.pdf'
test = parse_pdf(test_sentence_split)
print(test)
print('------')
test = spacy_split_sentences(test)
print(test)


# print('---------------')
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR04_22438994_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)


# annotations are in a list, should have results...
# But everything seems shifted quite a large amount. (relative to pages?, relative to parant?)
# print('---------------')
#test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR03_24806675_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)
# {'Dest': 'n33', 'Subtype': /Link, 'Type': /Annot, 'Border': [0, 0, 0], 'Rect': [195.003, 309.835, 293.202, 322.5]}
# many Dest
# {'C': [1.0, 1.0, 0.0], 'Popup': <PDFObjRef:386>, 'NM': 'c9e8becc-bb81-4687-81b2-74c7c475a53f', 'P': <PDFObjRef:323>, 'F': 4, 'Type': /Annot, 'M': "D:20151021133008-06'00'", 'Subtype': /Highlight, 'AP': {'N': <PDFObjRef:324>}, 'T': 'NAMI', 'QuadPoints': [329.524, 508.0, 374.55, 508.0, 329.524, 497.0, 374.55, 497.0], 'CreationDate': "D:20151021133008-06'00'", 'Subj': 'Highlight', 'Rect': [326.587, 496.656, 377.487, 508.344]}
# {'Parent': <PDFObjRef:385>, 'F': 28, 'Open': False, 'Subtype': /Popup, 'Type': /Annot, 'Rect': [594.0, 388.0, 774.0, 508.0]}
# [[329.524, 497.0, 374.55, 508.0], [197.297, 460.95, 553.187, 472.0], [51.4987, 449.0, 553.194, 460.0], [51.4987, 437.0, 288.023, 448.0]]

# try again, but more manually... observe works when we shift the bboxes.
#test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR03_24806675_tc_ANNOTATED.pdf'
## annot_boxes = [[329.524, 497.0, 374.55, 508.0], [197.297, 460.95, 553.187, 472.0], [51.4987, 449.0, 553.194, 460.0], [51.4987, 437.0, 288.023, 448.0]] # is off correct words, in fact no words.
# x = 9 # maybe the entire page's lower corner is shifted? Or the abstract is shifted?
# annot_boxes = [[329.524 - x, 497.0 - x, 374.55 - x, 508.0 - x], [197.297 - x, 460.95 - x, 553.187 - x, 472.0 - x], [51.4987 - x, 449.0 - x, 553.194 - x, 460.0 - x], [51.4987 - x, 437.0 - x, 288.023 - x, 448.0 - x]] # shifted, is on correct words
# fp = open(test_sentence_split, 'rb')
# parser = PDFParser(fp)
# document = PDFDocument(parser)
# if not document.is_extractable:
#     raise PDFTextExtractionNotAllowed
# rsrcmgr = PDFResourceManager()
# laparams = LAParams() # setting layout analysis params
# device = PDFPageAggregator(rsrcmgr, laparams=laparams)
# interpreter = PDFPageInterpreter(rsrcmgr, device)
# layouts = []
# for page in PDFPage.create_pages(document):
#     interpreter.process_page(page)
#     layouts.append(device.get_result())
# pages = list(PDFPage.get_pages(fp, set()))
# c = sort_all_chars_in_all_bboxes(layouts[0])
# page_result = extract_only_boxes(c, annot_boxes)
# print(page_result)

#annotations are in a list, but list works.
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR04_23941878_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)
# {'A': {'S': /URI, 'URI': 'http://dx.doi.org/10.1155/2014/627892'}, 'Subtype': /Link, 'C': [0, 0, 1], 'Border': [0, 0, 0], 'Rect': [50.59, 705.69, 171.72, 713.66]}
# {'C': [1.0, 1.0, 0.0], 'Popup': <PDFObjRef:305>, 'NM': 'eef900fb-9fe2-42f1-8f22-f95089bdb9d7', 'P': <PDFObjRef:269>, 'F': 4, 'Type': /Annot, 'M': "D:20151028154411-06'00'", 'Subtype': /Highlight, 'AP': {'N': <PDFObjRef:270>}, 'T': 'NAMI', 'QuadPoints': [313.351, 334.9, 339.238, 334.9, 313.351, 322.805, 339.238, 322.805], 'CreationDate': "D:20151028154411-06'00'", 'Subj': 'Highlight', 'Rect': [310.122, 322.427, 342.467, 335.278]}
# {'Parent': <PDFObjRef:304>, 'F': 28, 'Open': False, 'Subtype': /Popup, 'Type': /Annot, 'Rect': [600.05, 214.9, 780.05, 334.9]}
# ... C and Parent alternating

# annotations are in a list, but list works.
# test_sentence_split = './pdfs/all_pdfs_annotated_pmid_names/03_AR05_24980898_tc_ANNOTATED.pdf'
# test = parse_pdf(test_sentence_split)
# print(test)
# print('------')
# test = spacy_split_sentences(test)
# print(test)

INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP01_05_24333745_cc_ANNOTATED.pdf
DEBUG:root:Processing page : 0
DEBUG:root:
DEBUG:root:Processing page : 1
DEBUG:root:No annotations found for page : 1
DEBUG:root:



------




### handling the unicode
Seems easiest to handle unicode only in writing back the resulting parsed sentences. (i.e. single point of unicode intervention.)

Either:

with codecs.open(fout, 'w', 'utf-8') as f:
    ...
    f.write(...)

Or:

with open(fout, 'w') as f:
    sentences = spacy_split_sentences(v)
    f.write(...encode('utf-8'))

### A few test cases

Test files for 2 ways highlights may be organized in pdf.
 
    f = './some_test_pdfs/02_AG05_02_ANNOTATED_jl.pdf' # has a single  annotation obj resolving to list of annotation
  
    f = './some_test_pdfs/02_AP06_03_ANNOTATED_jl.pdf' # has a list of annotations

Test, includes bubble annotation

    f = './some_test_pdfs/02_LS05_01_24376698_cc_ANNOTATED.pdf # includes bubble annotation)

Test files for annotations on various pages

    f = './some_test_pdfs/03_AR01_25017671_tc_ANNOTATED.pdf # has annotations, but they are on page 2.
  
    f = './some_test_pdfs/03_AR01_21699821_tc_ANNOTATED.pdf # has  annotations, but split across page 1,2

TODO: Test files for italics.

    f = ./some_test_pdfs/02_AP06_03_ANNOTATED_mk.pdf (entirely drops n from n= )
  
    f = ./some_test_pdfs/02_AG05_02_ANNOTATED_jl.pdf (italics remain but are rearranged.)

TODO OR JUST CHECK EMPTY RESULTS: Test files for highlight or text has shifted coordinates.

    f = ./pdfs/all_pdfs_annotated_pmid_names/03_AR03_24806675_tc_ANNOTATED.pdf'
    

## Problem documents to annotate entirely by hand:

These were easily identified on OS X in Preview... 

    The highlights on these documents are obviously shifted when viewing as preview of document (but note: after opening in preview they look normal).
    
- Developmental Cognitive Neuroscience
  + 02_AR03_01_25261920
  + 02_AR03_03_25160677
- Neurobiology of disease
  + 03_AR03_24806675
- The Journal of Neuroscience
  + 02_AR00_02_25834059
  + 02_AR00_03_25788679
  + 02_LS05_02_20685988
  
Following have some other error... annotations in list.

  + _02_AP07_02_24804717_mk_ANNOTATED
  + _02_LS00_03_25913552_mk_ANNOTATED_IRRELEVANT
  
I had a bubble here, then tried to remove it, but failed?

  + _02_LS05_01_24376698_cc
  
Expected unicode, got string

  + _02_AR01_05_22379245

In [33]:
def get_files(path, annotator):
    # PUBMED ID in file name like: 03_AR05_23622762_mk_ANNOTATED
    f_pattern = path + '*_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_' + annotator + '*.pdf'
    files = glob.glob(f_pattern)
    return(files)

In [27]:
def parse_annotations_by_annotator(f_path_plus, result_dir, annotator):
    '''Set output file and write annotations, 1 per line 
    for this annotator for all files for this annotator'''
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    files = get_files(f_path_plus, annotator)
    results_d = {}
    for f in files:
        f_result = parse_pdf(f)
        results_d.update({f : f_result})
        # print(results_d)
    for k,v in results_d.items():
        bn = os.path.basename(k)
        bn = os.path.splitext(bn)[0] + '.txt'
        fout = os.path.join(result_dir, bn)
        with codecs.open(fout, 'w', 'utf-8') as f:
            print(bn)
            sentences = spacy_split_sentences(v)
            f.write(sentences)

## renamed 02 batched annotations towards their pubmed ids

In [115]:
%cd /Users/ccarey/Documents/Projects/NAMI/rdoc

logger.setLevel(logging.INFO)
parse_annotations_by_annotator(f_path_plus='./pdfs/all_pdfs_annotated_pmid_names/0', result_dir='./results/tc', annotator='tc')
parse_annotations_by_annotator(f_path_plus='./pdfs/all_pdfs_annotated_pmid_names/0', result_dir='./results/mk', annotator='mk')
# parse_annotations_by_annotator(f_path_plus='./pdfs/all_pdfs_annotated_pmid_names/0', result_dir='./results/jl', annotator='jl')
# parse_annotations_by_annotator(f_path_plus='./pdfs/all_pdfs_annotated_pmid_names/0', result_dir='./results/cc', annotator='cc')


INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP00_03_25773639_tc_IRRELEVANT.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP01_03_24740391_tc_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP07_05_21531705_tc_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP08_05_21957257_tc_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP11_03_23558179_tc_IRRELEVANT.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR00_05_25734385_tc_IRRELEVANT.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR01_01_23647728_tc_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR01_02_23083918_tc_IRRELEVANT.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR01_04_22575329_tc_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR01_05_22

/Users/ccarey/Documents/Projects/NAMI/rdoc
03_AR05_23622762_tc_ANNOTATED.txt
03_AR04_23709163_tc_ANNOTATED.txt
02_AR03_04_25136085_tc_ANNOTATED.txt
02_AR01_05_22379245_tc_ANNOTATED.txt
03_AR04_22379238_tc_ANNOTATED.txt
02_AP00_03_25773639_tc_IRRELEVANT.txt
02_AR04_05_24293773_tc_ANNOTATED.txt
03_AR04_23941878_tc_ANNOTATED.txt
02_AP07_05_21531705_tc_ANNOTATED.txt
03_AR01_20695690_tc_ANNOTATED.txt
03_AR01_25017671_tc_ANNOTATED.txt
02_LS08_04_24933724_tc_IRRELEVANT.txt
02_AR03_05_25126038_tc_ANNOTATED.txt
02_AR05_02_24101292_tc_ANNOTATED.txt
02_AP11_03_23558179_tc_IRRELEVANT.txt
03_AR03_24725811_tc_ANNOTATED.txt
02_LS00_05_25898427_tc_IRRELEVANT.txt
03_AR05_23088207_tc_ANNOTATED.txt
02_AR05_04_23954763_tc_ANNOTATED.txt
02_AR04_01_25142564_tc_ANNOTATED.txt
03_AR05_23143607_tc_ANNOTATED.txt
03_AR05_23074247_tc_ANNOTATED.txt
02_AR04_04_24333377_tc_ANNOTATED.txt
03_AR05_24770625_tc_ANNOTATED.txt
03_AR04_23904684_tc_ANNOTATED.txt
03_AR04_22438994_tc_ANNOTATED.txt
02_AR05_05_23646134_tc_ANNOTAT

INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AG05_01_23928891_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AG05_02_23744445_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP03_05_21613467_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP04_02_22447249_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP06_03_21319926_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP08_01_24023823_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AP11_02_24231418_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR00_05_25734385_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR01_01_23647728_mk_ANNOTATED.pdf
INFO:root:processing file : ./pdfs/all_pdfs_annotated_pmid_names/02_AR01_02_230839


03_AR04_23452958_tc_ANNOTATED.txt
03_AR01_21849230_tc_ANNOTATED.txt
02_LS06_02_25036160_mk_ANNOTATED_IRRELEVANT.txt
02_AP03_05_21613467_mk_ANNOTATED.txt
02_AR00_05_25734385_mk_ANNOTATED.txt
02_AR05_03_24045586_mk_ANNOTATED.txt
03_AR01_25017671_mk_ANNOTATED.txt
02_LS07_01_25280468_mk_ANNOTATED.txt
03_AR03_24725811_mk_ANNOTATED.txt
03_AR04_23452958_mk_ANNOTATED.txt
03_AR04_23709163_mk_ANNOTATED.txt
02_AP06_03_21319926_mk_ANNOTATED.txt
03_AR04_22438994_mk_ANNOTATED.txt
03_AR05_24770625_mk_ANNOTATED.txt
03_AR04_23904684_mk_ANNOTATED.txt
03_AR05_23143607_mk_ANNOTATED.txt
02_AR05_01_24116095_mk_ANNOTATED.txt
02_AG05_02_23744445_mk_ANNOTATED.txt
02_LS07_04_24388670_mk_ANNOTATED.txt
02_LS08_05_24359877_mk_ANNOTATED_IRRELEVANT.txt
02_AR05_05_23646134_mk_ANNOTATED.txt
02_AR01_04_22575329_mk_ANNOTATED.txt
02_AP11_02_24231418_mk_ANNOTATED.txt
03_AR01_21849230_mk_ANNOTATED.txt
03_AR05_23088207_mk_ANNOTATED.txt
03_AR05_23074247_mk_ANNOTATED.txt
02_AR03_02_25197810_mk_ANNOTATED.txt
03_AR01_20695690_

In [ ]:
# %cd /Users/ccarey/Documents/Projects/NAMI/rdoc/results
# ! for d in *; do echo $d; wc -l $d/*; done

### TODO: Need to distinguish positive (green) and negative (red) annotations.

### TODO: Need to edit results by hand to put in sentence endings and complete sentences as necessary.